In [2]:
from langchain_groq import ChatGroq

In [6]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='gsk_****', 
    model_name="llama-3.3-70b-specdec"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [9]:
pip install langchain-community

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ------------------------------------- -- 2.4/2.5 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 13.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 11.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19
Note: you may need to restart the kernel to use updated packages.


In [41]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.mercedes-benz.com/en/data-scientist-162735-MER0003L3Q")
page_data = loader.load().pop().page_content
print(page_data)





Data Scientist | Mercedes-Benz > Career > Job Search > Job Postings




































































 DE  EN Provider/PrivacyCompanyInnovationsSustainabilityCareersInvestorsPress ProductsCareersJob searchData ScientistJob category:IT / TelecommunicationsDepartment:MB Tech MalaysiaOrganization:Mercedes-Benz Malaysia Sdn. Bhd.Location:Mercedes-Benz Malaysia Sdn. Bhd., Kuala LumpurStart date:28.04.2025Publication date:14.03.2025Job number:MER0003L3QWorking time:full time ApplyTasksTask Description:
• Advising the customer on the use of AI, capturing the requirement and translating the requirement into suitable mathematical models• Data preparation, exploratory data analysis and statistical analysis. Conception of labelling processes• Selection, implementation and configuration of suitable (machine learning / semantic) models and procedures• Conception and implementation of automated pipelines for model creation and quality assurance• Conception and i

In [43]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    
    ### INSTRUCTION:
    Extract job postings and return them as a valid JSON object.
    
    The JSON should strictly follow this format:
    {{
        "role": "<Job Title>",
        "experience": "<Required Experience>",
        "skills": ["<Skill 1>", "<Skill 2>", "<Skill 3>", ...],
        "description": "<Job Description>"
    }}

    Ensure that:
    - The output is a **valid** JSON object (not a list unless there are multiple jobs).
    - No extra formatting, explanations, or markdown.
    - Strings in lists must be **properly comma-separated**.

    ### JSON OUTPUT:
    """
)


chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [45]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Data Scientist',
 'experience': 'Experience with Agile methodologies, Experience with Enterprise technologies, Experience with Product Oriented Teams, Programming or Data Architecture related certification',
 'skills': ['Excellent communication skills',
  'Good customer service orientation',
  'Analytical, passionate and drives technology and product quality',
  'Result-oriented thinking and action',
  'SQL/noSQL',
  'Python or R',
  'Spark',
  'Apache libraries',
  'Machine Learning algorithms',
  'Cloud Computing and Data Architectures'],
 'description': 'Advising the customer on the use of AI, capturing the requirement and translating the requirement into suitable mathematical models, Data preparation, exploratory data analysis and statistical analysis, Conception of labelling processes, Selection, implementation and configuration of suitable (machine learning / semantic) models and procedures, Conception and implementation of automated pipelines for model creation and qua

In [47]:
type(json_res)

dict

In [49]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [51]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [55]:
job = json_res

In [57]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/wordpress-p

In [59]:
job

{'role': 'Data Scientist',
 'experience': 'Experience with Agile methodologies, Experience with Enterprise technologies, Experience with Product Oriented Teams, Programming or Data Architecture related certification',
 'skills': ['Excellent communication skills',
  'Good customer service orientation',
  'Analytical, passionate and drives technology and product quality',
  'Result-oriented thinking and action',
  'SQL/noSQL',
  'Python or R',
  'Spark',
  'Apache libraries',
  'Machine Learning algorithms',
  'Cloud Computing and Data Architectures'],
 'description': 'Advising the customer on the use of AI, capturing the requirement and translating the requirement into suitable mathematical models, Data preparation, exploratory data analysis and statistical analysis, Conception of labelling processes, Selection, implementation and configuration of suitable (machine learning / semantic) models and procedures, Conception and implementation of automated pipelines for model creation and qua

In [61]:
job = json_res
job['skills']

['Excellent communication skills',
 'Good customer service orientation',
 'Analytical, passionate and drives technology and product quality',
 'Result-oriented thinking and action',
 'SQL/noSQL',
 'Python or R',
 'Spark',
 'Apache libraries',
 'Machine Learning algorithms',
 'Cloud Computing and Data Architectures']

In [63]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:  
        You are Karthik Mohan, a data analyst at Mercedes-Benz. Mercedes-Benz specializes in analyzing customer vehicle data to optimize component design, predict part failures, and enhance vehicle performance.  
        
        Your job is to write a cold email to the client regarding the job mentioned above, describing how your expertise in data analysis, AI, and predictive modeling can help them achieve their objectives.  
        
        Also, add the most relevant ones from the following links to showcase your work: {link_list}  
        
        Remember, you are Karthik Mohan, a data analyst at Mercedes-Benz.  
        Do not provide a preamble.  
        
        ### EMAIL (NO PREAMBLE):  

        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhancing Vehicle Performance with AI-Driven Data Analysis

Dear Hiring Manager,

I am excited to apply for the Data Scientist role, where I can leverage my expertise in data analysis, AI, and predictive modeling to drive business growth and optimize component design. As a data analyst at Mercedes-Benz, I have developed a strong passion for analyzing customer vehicle data to predict part failures and enhance vehicle performance.

With my excellent communication skills and customer service orientation, I am confident in advising clients on the use of AI and capturing requirements to translate them into suitable mathematical models. My analytical and result-oriented approach enables me to drive technology and product quality, ensuring compliance with applicable rules in data, AI, and IT security compliance.

My technical skills include proficiency in SQL/noSQL, Python, Spark, Apache libraries, and Machine Learning algorithms. I have experience with Cloud Computing and Data Archi